**Table of contents**<a id='toc0_'></a>    
- [Quick start](#toc1_)    
  - [Requirements](#toc1_1_)    
  - [SDK Description](#toc1_2_)    
    - [Package dependency description](#toc1_2_1_)    
    - [SDK installation](#toc1_2_2_)    
  - [API Host](#toc1_3_)    
    - [Test environment HTTP request list](#toc1_3_1_)    
    - [Test environment gRPC connection:](#toc1_3_2_)    
    - [Test environment MQTT connection:](#toc1_3_3_)    
- [API Application](#toc2_)    
  - [API Application](#toc2_1_)    
  - [Register an API Application](#toc2_2_)    
- [Examples of API calls](#toc3_)    
    - [Http interface list:](#toc3_1_1_)    
    - [Examples of trading pushes:](#toc3_1_2_)    
    - [Push list of market news:](#toc3_1_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [ ]:
from dotenv import load_dotenv, find_dotenv
import os

# Find and load the .env file
load_dotenv(find_dotenv())

# Access environment variables
WEBULL_TestAPPKEY = os.getenv("WEBULL_TestAPPKEY")
WEBULL_TestAPPSECRET = os.getenv("WEBULL_TestAPPSECRET")
WEBULL_ProdAPPKEY= os.getenv("WEBULL_ProdAPPKEY")
WEBULL_ProdAPPSECRET = os.getenv("WEBULL_ProdAPPSECRET")

# <a id='toc1_'></a>[Quick start](#toc0_)

## <a id='toc1_1_'></a>[Requirements](#toc0_)

## <a id='toc1_2_'></a>[SDK Description](#toc0_)

### <a id='toc1_2_1_'></a>[Package dependency description](#toc0_)
### <a id='toc1_2_2_'></a>[SDK installation](#toc0_)

## <a id='toc1_3_'></a>[API Host](#toc0_)

### <a id='toc1_3_1_'></a>[Test environment HTTP request list](#toc0_)

In [ ]:
import uuid

from webullsdkcore.client import ApiClient
from webullsdktrade.api import API
from webullsdkcore.common.region import Region


In [ ]:

optional_api_endpoint = "hk-openapi.uat.webullbroker.com"
your_app_key = WEBULL_TestAPPKEY
your_app_secret = WEBULL_TestAPPSECRET


In [ ]:
api_client = ApiClient(your_app_key, your_app_secret, Region.HK.value)
api_client.add_endpoint(Region.HK.value, optional_api_endpoint)


In [ ]:


class TestOrder(object):
    def __init__(self):
        self.api = API(api_client)

    def order(self):
        res = self.api.account.get_app_subscriptions()
        account_id = None
        if res.status_code == 200:
            print('app subscriptions:', res.json())
            result = res.json()
            account_id = result[0]['account_id']
            print("account id:", account_id)

        if account_id is None:
            print("account id is null")
            return

        client_order_id = uuid.uuid4().hex
        print('client order id:', client_order_id)
        stock_order = {
            "account_id": account_id,
            "stock_order": {
                "client_order_id": client_order_id,
                "instrument_id": "913256409",
                "side": "BUY",
                "tif": "DAY",
                "order_type": "AT_AUCTION",
                # "limit_price": "385.000",
                "qty": "1",
                "extended_hours_trading": False
            }
        }
        res = self.api.order.place_order(stock_order['account_id'], **stock_order['stock_order'])
        if res.status_code == 200:
            print('place order res:', res.json())


if __name__ == '__main__':
    openapi = TestOrder()
    openapi.order()

### <a id='toc1_3_2_'></a>[Test environment gRPC connection:](#toc0_)

In [ ]:
import logging

from webullsdkcore.client import ApiClient
from webullsdktradeeventscore.events_client import EventsClient
from webullsdktrade.api import API
from webullsdkcore.common.region import Region

optional_api_endpoint = "hk-openapi.uat.webullbroker.com"
your_app_key = WEBULL_TestAPPKEY
your_app_secret = WEBULL_TestAPPSECRET
api_client = ApiClient(your_app_key, your_app_secret, Region.HK.value)
api_client.add_endpoint(Region.HK.value, optional_api_endpoint)
PRE_HOST = "hk-openapi-events-api.uat.webullbroker.com"


def _on_log(level, log_content):
    print(logging.getLevelName(level), log_content)


if __name__ == '__main__':
    api = API(api_client)
    res = api.account.get_app_subscriptions()
    account_id = None
    if res.status_code == 200:
        print('app subscriptions:', res.json())
        result = res.json()
        account_id = result[0]['account_id']
        print("account id:", account_id)
    if account_id:
        client = EventsClient(your_app_key, your_app_secret, host=PRE_HOST)
        client.on_log = _on_log
        print("subscribe account", [account_id])
        client.do_subscribe([account_id])
    else:
        print("account id is null")

### <a id='toc1_3_3_'></a>[Test environment MQTT connection:](#toc0_)

In [ ]:
from webullsdkmdata.common.category import Category
from webullsdkmdata.common.subscribe_type import SubscribeType
from webullsdkmdata.quotes.subscribe.default_client import DefaultQuotesClient
from webullsdkcore.common.region import Region

your_app_key = WEBULL_TestAPPKEY
your_app_secret = WEBULL_TestAPPSECRET
optional_quotes_endpoint = "hk-openapi-quotes-api.uat.webullbroker.com"

def pt_logs(client, userdata, level, buf):
    print("userdata:%s, level:%s, buf:%s" % (userdata, level, buf))


def on_message(client, userdata, message):
    print("Received message '" + str(message.payload) + "' on topic '"
          + message.topic + "' with QoS " + str(message.qos))


if __name__ == '__main__':
    client = DefaultQuotesClient(your_app_key, your_app_secret, Region.HK.value, optional_quotes_endpoint)
    client.init_default_settings('00700', Category.HK_STOCK.name, SubscribeType.SNAPSHOT.name)
    client.on_log = pt_logs
    client.on_message = on_message
    client.connect_and_loop_forever()


# <a id='toc2_'></a>[API Application](#toc0_)
## <a id='toc2_1_'></a>[API Application](#toc0_)

## <a id='toc2_2_'></a>[Register an API Application](#toc0_)




# <a id='toc3_'></a>[Examples of API calls](#toc0_)

### <a id='toc3_1_1_'></a>[Http interface list:](#toc0_)

In [ ]:
from webullsdkcore.client import ApiClient
from webullsdktrade.api import API
from webullsdkcore.common.region import Region

your_app_key = WEBULL_ProdAPPKEY
your_app_secret = WEBULL_ProdAPPSECRET
api_client = ApiClient(your_app_key, your_app_secret, Region.HK.value)
api = API(api_client)
if __name__ == '__main__':
    res = api.account.get_app_subscriptions()
    account_id = None
    if res.status_code == 200:
        print('app subscriptions:', res.json())
        result = res.json()
        account_id = result[0]['account_id']
        print("account id:", account_id)

    if account_id is None:
        print("account id is null")

### <a id='toc3_1_2_'></a>[Examples of trading pushes:](#toc0_)

In [ ]:
account_id

In [ ]:
import logging

from webullsdktradeeventscore.events_client import EventsClient
from webullsdkcore.common.region import Region

your_app_key = WEBULL_ProdAPPKEY
your_app_secret = WEBULL_ProdAPPSECRET
# account_id = "<your_account_id>"


def _on_log(level, log_content):
    print(logging.getLevelName(level), log_content)


if __name__ == '__main__':
    client = EventsClient(your_app_key, your_app_secret,Region.HK.value)
    client.on_log = _on_log
    print("subscribe account", [account_id])
    client.do_subscribe([account_id])

### <a id='toc3_1_3_'></a>[Push list of market news:](#toc0_)

In [ ]:
from webullsdkmdata.common.category import Category
from webullsdkmdata.common.subscribe_type import SubscribeType
from webullsdkmdata.quotes.subscribe.default_client import DefaultQuotesClient
from webullsdkcore.common.region import Region

your_app_key = WEBULL_ProdAPPKEY
your_app_secret = WEBULL_ProdAPPSECRET


def pt_logs(client, userdata, level, buf):
    print("userdata:%s, level:%s, buf:%s" % (userdata, level, buf))


def on_message(client, userdata, message):
    print("Received message '" + str(message.payload) + "' on topic '"
          + message.topic + "' with QoS " + str(message.qos))


if __name__ == '__main__':
    client = DefaultQuotesClient(your_app_key, your_app_secret, Region.HK.value)
    client.init_default_settings('00700', Category.HK_STOCK.name, SubscribeType.SNAPSHOT.name)
    client.on_log = pt_logs
    client.on_message = on_message
    client.connect_and_loop_forever()